#### Import Libraries

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import SGD
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData, AttackResultsCollection, AttackType, PrivacyMetric, PrivacyReportMetadata, SlicingSpec
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack import privacy_report

#### Data Preprocessing

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# Load the dataset
data = pd.read_csv('../data/patient_processed.csv')

In [14]:
# Select the relevant columns
features = data[['tanimoto', 'feature_jsim', 'feature_dsim', 'feature_osim']]
labels = data['DDI']

# Map DDI levels to the numeric values
ddi_mapping = {'no interaction found': 0, 'minor': 1, 'moderate': 2, 'major': 3}
labels = labels.map(ddi_mapping)

In [15]:
# Handle missing values
features = features.fillna(features.mean())

In [16]:
# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    labels, 
    test_size=0.2, 
    random_state=42
)

In [17]:
# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
# Convert the target variable to a categorical format
num_classes = len(ddi_mapping)
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

#### Define Hyperparameters

In [19]:
# Model Hypeparameters
num_features = len(ddi_mapping)
num_classes = X_train_scaled.shape[1]
learning_rate = 0.01
batch_size = 32
epochs = 10

# Privacy Hyperparameters
l2_norm_clip = 1.0
noise_multiplier = 1.1
num_microbatches = 1

#### Build and Compile the Models

##### Model Architecture

In [20]:
import tensorflow as tf
from tensorflow import keras


# Define the model architecture
model = keras.Sequential([
    keras.layers.Input(shape=(num_features,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

##### Differential Privacy Model

In [22]:
# Build a model with differential privacy
model_dp = model

# Build the DP-SGD Optimizer
optimizer_dp = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
)

# Compile the model
model_dp.compile(
    optimizer=optimizer_dp,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
# Train the model
history = model_dp.fit(
    X_train_scaled, y_train_cat,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/100
5/5 [==============================] - 4s 111ms/step - loss: 0.7575 - accuracy: 0.7442 - val_loss: 0.5747 - val_accuracy: 0.8788
Epoch 2/100
5/5 [==============================] - 0s 27ms/step - loss: 0.7579 - accuracy: 0.7442 - val_loss: 0.5842 - val_accuracy: 0.8788
Epoch 3/100
5/5 [==============================] - 0s 26ms/step - loss: 0.7577 - accuracy: 0.7442 - val_loss: 0.5737 - val_accuracy: 0.8788
Epoch 4/100
5/5 [==============================] - 0s 26ms/step - loss: 0.7629 - accuracy: 0.7442 - val_loss: 0.5468 - val_accuracy: 0.8788
Epoch 5/100
5/5 [==============================] - 0s 25ms/step - loss: 0.7541 - accuracy: 0.7442 - val_loss: 0.5783 - val_accuracy: 0.8788
Epoch 6/100
5/5 [==============================] - 0s 25ms/step - loss: 0.7896 - accuracy: 0.7442 - val_loss: 0.5912 - val_accuracy: 0.8485
Epoch 7/100
5/5 [==============================] - 0s 25ms/step - loss: 0.7687 - accuracy: 0.7442 - val_loss: 0.5705 - val_accuracy: 0.8485
Epoch 8/100
5/5 [==

##### Non-Differential Privacy Model

In [13]:
from keras.optimizers import SGD

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

optimizer_dp = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
)

loss_dp = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)

model_dp.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9), loss=loss_dp, metrics=['accuracy'])

# Train the DP model
model_dp.fit(X_train, y_train, epochs=10, validation_data=(X_train, y_train), batch_size=32)

Epoch 1/10


c:\MONASH\Y3S2\FIT3164\repo\MDS14\.venv\Lib\site-packages\keras\src\backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


4/4 [==============================] - 1s 79ms/step - loss: 0.9976 - accuracy: 0.7477 - val_loss: 0.9142 - val_accuracy: 0.7477
Epoch 2/10
4/4 [==============================] - 0s 20ms/step - loss: 0.8618 - accuracy: 0.7477 - val_loss: 0.8698 - val_accuracy: 0.7477


c:\MONASH\Y3S2\FIT3164\repo\MDS14\.venv\Lib\site-packages\keras\src\backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 3/10
4/4 [==============================] - 0s 22ms/step - loss: 0.9042 - accuracy: 0.7477 - val_loss: 0.8240 - val_accuracy: 0.7477
Epoch 4/10
4/4 [==============================] - 0s 20ms/step - loss: 0.8530 - accuracy: 0.7477 - val_loss: 0.8421 - val_accuracy: 0.7477
Epoch 5/10
4/4 [==============================] - 0s 22ms/step - loss: 0.8419 - accuracy: 0.7477 - val_loss: 0.8354 - val_accuracy: 0.7477
Epoch 6/10
4/4 [==============================] - 0s 25ms/step - loss: 0.8570 - accuracy: 0.7477 - val_loss: 0.8169 - val_accuracy: 0.7477
Epoch 7/10
4/4 [==============================] - 0s 21ms/step - loss: 0.8081 - accuracy: 0.7477 - val_loss: 0.8361 - val_accuracy: 0.7477
Epoch 8/10
4/4 [==============================] - 0s 19ms/step - loss: 0.8319 - accuracy: 0.7477 - val_loss: 0.8129 - val_accuracy: 0.7477
Epoch 9/10
4/4 [==============================] - 0s 21ms/step - loss: 0.8095 - accuracy: 0.7477 - val_loss: 0.8077 - val_accuracy: 0.7477
Epoch 10/10
4/4 [==========

#### Measuring the differential privacy guarantee

In [28]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy_lib

epsilon, best_alpha = compute_dp_sgd_privacy_lib.compute_dp_sgd_privacy(
    n=X_train.shape[0],
    batch_size=batch_size,
    noise_multiplier=noise_multiplier,
    epochs=epochs,
    delta=1e-5
)

print(f"Epsilon: {epsilon}")
print(f"Best alpha: {best_alpha}")

Epsilon: 11.593025381592284
Best alpha: 2.5


#### Define Callback for Privacy Metrics

In [14]:
class PrivacyMetrics(tf.keras.callbacks.Callback):
    def __init__(self, epochs_per_report, model_name):
        self.epochs_per_report = epochs_per_report
        self.model_name = model_name
        self.attack_results = []

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.epochs_per_report != 0:
            return

        logits_train = self.model.predict(X_train, batch_size=batch_size)
        logits_test = self.model.predict(X_test, batch_size=batch_size)

        prob_train = tf.nn.softmax(logits_train, axis=1)
        prob_test = tf.nn.softmax(logits_test, axis=1)

        privacy_report_metadata = PrivacyReportMetadata(
            accuracy_train=logs['accuracy'],
            accuracy_test=logs['val_accuracy'],
            epoch_num=epoch + 1,
            model_variant_label=self.model_name
        )

        attack_results = mia.run_attacks(
            AttackInputData(
                labels_train=y_train,
                labels_test=y_test,
                probs_train=prob_train,
                probs_test=prob_test
            ),
            SlicingSpec(entire_dataset=True, by_class=True),
            attack_types=(AttackType.THRESHOLD_ATTACK, AttackType.LOGISTIC_REGRESSION),
            privacy_report_metadata=privacy_report_metadata
        )

        self.attack_results.append(attack_results)

#### Train Model and Collect Privacy Metrics

In [15]:
callback = PrivacyMetrics(epochs_per_report=2, model_name="DP Model")
history = model_dp.fit(
    X_train, y_train,
    epochs=epochs,
    validation_data=(X_test, y_test),
    batch_size=batch_size,
    callbacks=[callback]
)

Epoch 1/10
4/4 [==============================] - 0s 33ms/step - loss: 0.7836 - accuracy: 0.7477 - val_loss: 1.0185 - val_accuracy: 0.6071
Epoch 2/10
1/1 [==============================] - 0s 24ms/step


ValueError: probs_train should be a numpy array.

#### Visualise Privacy Metrics

In [ ]:
all_reports = callback.attack_results
results = AttackResultsCollection(all_reports)

##### Epoch Plots

In [ ]:
privacy_metrics = (PrivacyMetric.AUC, PrivacyMetric.ATTACKER_ADVANTAGE)
epoch_plot = privacy_report.plot_by_epochs(results, privacy_metrics=privacy_metrics)

##### Privacy VS Utility Plots

In [ ]:
utility_privacy_plot = privacy_report.plot_privacy_vs_accuracy(results, privacy_metrics=privacy_metrics)
for axis in utility_privacy_plot.axes:
    axis.set_xlabel('Validation accuracy')